# 作業一

In [162]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import math
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 6)
pd.set_option('display.float_format', lambda x : '%.4f' % x)

In [163]:
#讀HPRICE2資料
hprice2= pd.read_csv("hprice2.csv")
hprice2.head(3)

,price,crime,nox,rooms,dist,radial,proptax,stratio,lowstat,lprice,lnox,lproptax
0,24000,0.0060,5.3800,6.5700,4.0900,1,29.6000,15.3000,4.9800,10.0858,1.6827,5.6904
1,21599,0.0270,4.6900,6.4200,4.9700,2,24.2000,17.8000,9.1400,9.9804,1.5454,5.4889
2,34700,0.0270,4.6900,7.1800,4.9700,2,24.2000,17.8000,4.0300,10.4545,1.5454,5.4889


In [164]:
# y 
lprice = hprice2['lprice']
price = hprice2['price']

# x
ldist = np.log(hprice2[['dist']])
ldist.columns = ['ldist'] 
pairf_1 = pd.concat([hprice2[['lnox', 'rooms', 'stratio']], ldist], axis = 1)
pairf_2 = hprice2[['nox', 'dist', 'rooms', 'stratio']]

## (1) 先個別估計兩條迴歸式。

### 1. 取log
<img src="https://i.imgur.com/4kA2tzv.png" width="70%">

In [165]:
#1. log(price)
model_1 = sm.OLS(lprice, sm.add_constant(pairf_1)).fit()
print(model_1.summary())

                            OLS Regression Results                            
Dep. Variable:                 lprice   R-squared:                       0.584
Model:                            OLS   Adj. R-squared:                  0.581
Method:                 Least Squares   F-statistic:                     175.9
Date:                Sat, 12 Jun 2021   Prob (F-statistic):           5.53e-94
Time:                        00:50:47   Log-Likelihood:                -43.495
No. Observations:                 506   AIC:                             96.99
Df Residuals:                     501   BIC:                             118.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         11.0839      0.318     34.843      0.0

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [166]:
# SST 總平方和
coef_log = model_1.params
y_log = coef_log['const'] + coef_log['lnox'] * pairf_1['lnox'] + coef_log['ldist'] * pairf_1['ldist'] \
                          + coef_log['rooms'] * pairf_1['rooms'] + coef_log['stratio'] * pairf_1['stratio'] 

SST = sum((lprice - np.mean(y_log))**2)
print('lprice 的 SST 為', SST)

lprice 的 SST 為 84.58222496907344


### 2. 沒取log

<img src="https://i.imgur.com/3DL3PHb.png" width="60%">

In [167]:
#2. price
model_2 = sm.OLS(price, sm.add_constant(pairf_2)).fit()
print(model_2.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.620
Model:                            OLS   Adj. R-squared:                  0.617
Method:                 Least Squares   F-statistic:                     204.2
Date:                Sat, 12 Jun 2021   Prob (F-statistic):          9.63e-104
Time:                        00:50:47   Log-Likelihood:                -5091.5
No. Observations:                 506   AIC:                         1.019e+04
Df Residuals:                     501   BIC:                         1.021e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.372e+04   5120.564      4.632      0.0

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [168]:
# SST 總平方和
coef = model_2.params
y_nolog = coef['const'] + coef['nox'] * pairf_2['nox'] + coef['dist'] * pairf_2['dist'] \
                    + coef['rooms'] * pairf_2['rooms'] + coef['stratio'] * pairf_2['stratio'] 

SST = sum((price - np.mean(y_nolog))**2)
print('price 的 SST 為', SST)

price 的 SST 為 42825531146.45058


### 3. 小結
                  R2        SST
    -------------------------------
    log_price    0.584     84.58
    price        0.620  42825531146

## (2) 帶入數值求算預測房價。 nox=5.55; dist=3.80; rooms=6.28; stratio=18.46

In [169]:
#找 nox=5.55; dist=3.80; rooms=6.28; stratio=18.46 時的 log_price 預測值
y_pre = coef_log['const'] + coef_log['lnox'] * np.log(5.55) + coef_log['ldist'] * np.log(3.8) \
                          + coef_log['rooms'] * 6.28 + coef_log['stratio'] * 18.46
y_pre

9.90052739938465

In [170]:
# y_log = log_price的預測值
m = np.exp(y_log)
m

0     25844.9422
1     24227.9612
2     29398.6247
         ...    
503   21813.9013
504   20516.1962
505   16796.8761
Length: 506, dtype: float64

In [171]:
# 迴歸分析  #無截距項
model_3 = sm.OLS(price, m).fit()
print(model_3.summary())

                                 OLS Regression Results                                
Dep. Variable:                  price   R-squared (uncentered):                   0.951
Model:                            OLS   Adj. R-squared (uncentered):              0.950
Method:                 Least Squares   F-statistic:                              9709.
Date:                Sat, 12 Jun 2021   Prob (F-statistic):                        0.00
Time:                        00:50:47   Log-Likelihood:                         -5067.3
No. Observations:                 506   AIC:                                  1.014e+04
Df Residuals:                     505   BIC:                                  1.014e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [172]:
# M 的 coef = 1.0298
price_1 = model_3.params[0] * math.exp(y_pre)
price_1

20535.808119134777

In [173]:
#R-square為實際y值與配飾值y的相關係數平方
r = hprice2['price'].corr(m)
r

0.8090527455431987

In [174]:
rr = r**2
rr

0.6545663450709879

## (3) 利用轉換進行比較哪一個迴歸式比較好?

    log(price)經過轉換之後，跟price一樣的單位，r-squared會 = 0.65
    price 的 r-squared = 0.62，故log(price)經過轉換之後的r-squared較大
    => 取log模型比較好